In [8]:
import pandas as pd
import numpy as np
import os
import dotenv


import re
from openai import AzureOpenAI



dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')


# AZURE_OPEN_AI_ORGANIZATION = os.getenv('AZURE_OPEN_AI_ORGANIZATION')



In [9]:
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s


In [10]:
data = pd.read_parquet('/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/notebooks/splitted.parquet')

data['chunk']= data["chunk"].apply(lambda x : normalize_text(x))
data

,filename,document_sha,chunk_sha,chunk
0,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c001f008-b5df-5ec0-8905-17ae645171e6,Empanadas argentinas
1,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,8830dfd7-1331-5221-8a08-5f8cc7c46c49,Ingredientes Medidas 4 Porciones Seleccionar t...
2,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,9c95627f-1e0c-5a5d-8bdd-eb4e9cbb7da0,"1 taza de cebolla, finamente picada, para el r..."
3,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b88038c3-6087-52c9-a2df-dffef9bff309,"1 taza de huevo duro, finamente picado, para e..."
4,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,1f433cc2-34d7-511d-addd-f8660e850f36,"suficiente de harina, para extender la masa su..."
5,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c60774cf-e659-5237-a63e-3bd0aea3ccfb,Para el relleno de carne: calienta el aceite d...
6,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b839d3a7-4daf-5460-975c-cb76ee3dc321,Para el relleno de elote: derrite la mantequil...
7,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,e554b863-d17a-588f-9210-b05955c6f34b,Barniza las empanadas argentinas con el huevo ...
8,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,54af8361-016d-565e-9803-dccbc11fb95d,Pavo a la hawaiana Ingredientes 1 taza de sals...
9,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,367213bc-c570-5935-be25-ed7993c225b8,"Preparación En un bowl, combina la salsa de so..."


In [11]:
client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

In [12]:
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [13]:
data['vector'] = data["chunk"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model shou